# NFL Prospect Analysis Notebook

This notebook demonstrates how to use the Draft Queen API for exploratory analysis of NFL draft prospects.

## Overview
- Query prospects with complex filters
- Analyze position-specific statistics
- Export data for further analysis
- Visualize prospect distributions
- Compare prospects across positions

## Setup

Make sure the API server is running on http://localhost:8000

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime

# API configuration
API_BASE = "http://localhost:8000"
API_PROSPECTS = f"{API_BASE}/api/prospects"
API_ANALYTICS = f"{API_BASE}/api/analytics"
API_EXPORTS = f"{API_BASE}/api/exports"

print(f"API Base URL: {API_BASE}")
print(f"Testing connection...")

try:
    response = requests.get(f"{API_BASE}/health", timeout=5)
    if response.status_code == 200:
        print(f"✓ API is healthy")
    else:
        print(f"✗ API returned status {response.status_code}")
except Exception as e:
    print(f"✗ Failed to connect to API: {e}")

## 1. Basic Prospect Queries

Let's start by fetching all prospects and exploring the data.

In [ ]:
# Fetch all prospects
response = requests.get(f"{API_PROSPECTS}/", params={"limit": 100})
data = response.json()

print(f"Response status: {response.status_code}")
print(f"Query execution time: {data.get('execution_time_ms', 'N/A')}ms")
print(f"Total prospects: {data.get('total_count', 0)}")
print(f"Records returned: {data.get('record_count', 0)}")

# Convert to DataFrame
prospects_df = pd.DataFrame(data['prospects'])
print(f"\nDataFrame shape: {prospects_df.shape}")
print(f"\nColumns: {list(prospects_df.columns)}")
print(f"\nFirst few records:")
prospects_df.head()

## 2. Query by Position

Filter prospects by position (QB, RB, WR, etc.)

In [ ]:
# Query QBs only
response = requests.get(f"{API_PROSPECTS}/", params={"position": "QB", "limit": 50})
qb_data = response.json()
qb_df = pd.DataFrame(qb_data['prospects'])

print(f"QBs found: {qb_data.get('record_count', 0)}")
print(f"\nQB Data Summary:")
print(qb_df[['name', 'position', 'college', 'height', 'weight', 'draft_grade']].head(10))

# Basic statistics
print(f"\nQB Statistics:")
print(qb_df[['height', 'weight', 'draft_grade']].describe())

## 3. Complex Filtering

Use the advanced query API with multiple filters combined with AND logic.

In [ ]:
# Complex query: QBs from top colleges with height 6.0-6.5
query_payload = {
    "position": "QB",
    "height_min": 6.0,
    "height_max": 6.5,
    "draft_grade_min": 7.5,
    "limit": 50
}

response = requests.post(f"{API_PROSPECTS}/query", json=query_payload)
filtered_data = response.json()
filtered_df = pd.DataFrame(filtered_data['prospects'])

print(f"Query payload: {json.dumps(query_payload, indent=2)}")
print(f"\nResults:")
print(f"  - Matched: {filtered_data.get('record_count', 0)} prospects")
print(f"  - Execution time: {filtered_data.get('execution_time_ms', 'N/A')}ms")
print(f"  - Query hash: {filtered_data.get('query_hash', 'N/A')}")

print(f"\nMatched prospects:")
print(filtered_df[['name', 'position', 'college', 'height', 'weight', 'draft_grade']].to_string())

## 4. Position Analytics

Get detailed statistics for a specific position including percentiles.

In [ ]:
# Get QB position statistics
response = requests.get(f"{API_ANALYTICS}/positions/QB")
stats = response.json()

print(f"Position: {stats['position']}")
print(f"Count: {stats['count']}")
print(f"\nHeight Statistics (feet):")
if stats['height']:
    for key, value in stats['height'].items():
        print(f"  {key}: {value}")

print(f"\nWeight Statistics (lbs):")
if stats['weight']:
    for key, value in stats['weight'].items():
        print(f"  {key}: {value}")

print(f"\nDraft Grade Statistics:")
if stats['draft_grade']:
    for key, value in stats['draft_grade'].items():
        print(f"  {key}: {value}")

## 5. All Positions Summary

Get a summary of all positions at once.

In [ ]:
# Get all positions summary
response = requests.get(f"{API_ANALYTICS}/positions")
all_positions = response.json()

print(f"Total positions: {all_positions['total_positions']}")
print(f"\nPosition Summary:")

# Convert to DataFrame for better visualization
positions_df = pd.DataFrame.from_dict(all_positions['positions'], orient='index')
positions_df.index.name = 'Position'
print(positions_df.to_string())

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Count by position
positions_df['count'].plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Prospect Count by Position')
axes[0, 0].set_ylabel('Count')

# Average height
positions_df['height_avg'].plot(kind='bar', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Average Height by Position')
axes[0, 1].set_ylabel('Height (feet)')

# Average weight
positions_df['weight_avg'].plot(kind='bar', ax=axes[1, 0], color='lightgreen')
axes[1, 0].set_title('Average Weight by Position')
axes[1, 0].set_ylabel('Weight (lbs)')

# Average draft grade
positions_df['draft_grade_avg'].plot(kind='bar', ax=axes[1, 1], color='gold')
axes[1, 1].set_title('Average Draft Grade by Position')
axes[1, 1].set_ylabel('Draft Grade')

plt.tight_layout()
plt.show()

## 6. Distribution Analysis

Analyze the distribution of prospect attributes.

In [ ]:
# Get all prospects for distribution analysis
response = requests.get(f"{API_PROSPECTS}/", params={"limit": 100})
all_data = response.json()
all_df = pd.DataFrame(all_data['prospects'])

# Create distribution plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Height distribution
all_df['height'].hist(bins=20, ax=axes[0, 0], color='steelblue', edgecolor='black')
axes[0, 0].set_title('Height Distribution (All Prospects)')
axes[0, 0].set_xlabel('Height (feet)')
axes[0, 0].set_ylabel('Frequency')

# Weight distribution
all_df['weight'].hist(bins=20, ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_title('Weight Distribution (All Prospects)')
axes[0, 1].set_xlabel('Weight (lbs)')
axes[0, 1].set_ylabel('Frequency')

# Draft grade distribution
all_df['draft_grade'].hist(bins=15, ax=axes[1, 0], color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Draft Grade Distribution (All Prospects)')
axes[1, 0].set_xlabel('Draft Grade')
axes[1, 0].set_ylabel('Frequency')

# Prospects by position
position_counts = all_df['position'].value_counts()
position_counts.plot(kind='barh', ax=axes[1, 1], color='gold')
axes[1, 1].set_title('Prospect Count by Position')
axes[1, 1].set_xlabel('Count')

plt.tight_layout()
plt.show()

print(f"\nBasic Statistics:")
print(all_df[['height', 'weight', 'draft_grade']].describe())

## 7. Position Comparisons

Compare attributes across different positions.

In [ ]:
# Get all prospects
response = requests.get(f"{API_PROSPECTS}/", params={"limit": 100})
comparison_df = pd.DataFrame(response.json()['prospects'])

# Calculate position-specific statistics
position_stats = comparison_df.groupby('position')[['height', 'weight', 'draft_grade']].agg(['mean', 'std', 'min', 'max', 'count'])

print("Position-wise Statistics:")
print(position_stats)

# Visualization: Position comparisons
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Height by position (box plot)
height_by_pos = [comparison_df[comparison_df['position'] == pos]['height'].dropna().values 
                   for pos in comparison_df['position'].unique()]
axes[0].boxplot(height_by_pos, labels=comparison_df['position'].unique())
axes[0].set_title('Height Distribution by Position')
axes[0].set_ylabel('Height (feet)')
axes[0].tick_params(axis='x', rotation=45)

# Weight by position (box plot)
weight_by_pos = [comparison_df[comparison_df['position'] == pos]['weight'].dropna().values 
                  for pos in comparison_df['position'].unique()]
axes[1].boxplot(weight_by_pos, labels=comparison_df['position'].unique())
axes[1].set_title('Weight Distribution by Position')
axes[1].set_ylabel('Weight (lbs)')
axes[1].tick_params(axis='x', rotation=45)

# Draft grade by position (box plot)
grade_by_pos = [comparison_df[comparison_df['position'] == pos]['draft_grade'].dropna().values 
                 for pos in comparison_df['position'].unique()]
axes[2].boxplot(grade_by_pos, labels=comparison_df['position'].unique())
axes[2].set_title('Draft Grade Distribution by Position')
axes[2].set_ylabel('Draft Grade')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Export Data

Export the data for use in other tools.

In [ ]:
# Export as JSON
export_payload = {
    "format": "json",
    "filters": {
        "limit": 50
    },
    "pretty": True
}

response = requests.post(f"{API_EXPORTS}/", json=export_payload)
print(f"Export status: {response.status_code}")
print(f"Content-Type: {response.headers.get('Content-Type')}")
print(f"Content-Length: {response.headers.get('Content-Length')} bytes")

# Save to file
with open('prospects_export.json', 'w') as f:
    f.write(response.text)
print(f"\n✓ Exported to prospects_export.json")

# Export as CSV
response = requests.get(f"{API_EXPORTS}/csv", params={"limit": 50})
print(f"\nCSV export status: {response.status_code}")
print(f"Content-Type: {response.headers.get('Content-Type')}")
print(f"First 500 chars of CSV:")
print(response.text[:500])

## 9. Advanced Analysis Examples

More sophisticated analysis combining multiple queries.

In [ ]:
# Example 1: Find QB prospects with above-average draft grades
response = requests.get(f"{API_PROSPECTS}/", params={"limit": 100})
all_prospects = pd.DataFrame(response.json()['prospects'])

overall_avg_grade = all_prospects['draft_grade'].mean()
top_qbs = all_prospects[(all_prospects['position'] == 'QB') & 
                        (all_prospects['draft_grade'] > overall_avg_grade)]

print(f"Overall average draft grade: {overall_avg_grade:.2f}")
print(f"\nTop-rated QBs (above average):")
print(top_qbs[['name', 'college', 'draft_grade']].sort_values('draft_grade', ascending=False).to_string())

# Example 2: Identify height/weight outliers by position
print(f"\n\nHeight/Weight Outliers:")
for position in all_prospects['position'].unique():
    pos_data = all_prospects[all_prospects['position'] == position]
    if len(pos_data) > 0:
        height_mean = pos_data['height'].mean()
        height_std = pos_data['height'].std()
        outliers = pos_data[abs(pos_data['height'] - height_mean) > 2*height_std]
        if len(outliers) > 0:
            print(f"\n{position}:")
            print(f"  Average height: {height_mean:.2f}ft (±{height_std:.2f})")
            print(f"  Outliers: {len(outliers)}")
            for _, row in outliers.iterrows():
                print(f"    - {row['name']}: {row['height']:.2f}ft")

## 10. Summary

This notebook demonstrated:
- **Basic queries**: Fetching all prospects and filtering by position
- **Advanced filtering**: Using complex queries with multiple criteria
- **Analytics**: Getting position-specific statistics and percentiles
- **Visualizations**: Creating plots and distributions
- **Comparisons**: Analyzing differences across positions
- **Data export**: Exporting results in multiple formats
- **Advanced analysis**: Finding patterns and outliers

You can use these techniques to:
- Identify top prospects by position
- Compare college performance
- Analyze physical attributes
- Find data anomalies
- Create custom reports

## Appendix: Useful API Endpoints

### Query Prospects
- `GET /api/prospects/?position=QB&limit=10` - Get QBs with limit
- `POST /api/prospects/query` - Advanced queries with filters

### Analytics
- `GET /api/analytics/positions/{position}` - Get position statistics
- `GET /api/analytics/positions` - Get all positions summary

### Export
- `POST /api/exports/` - Export with format and filters
- `GET /api/exports/{format}` - Quick export

### Health
- `GET /health` - Check API status
- `GET /docs` - OpenAPI documentation